In [1]:
import thrift
import sys
import pprint
if sys.version_info[0] > 2:
    from urllib.parse import urlparse
else:
    from urlparse import urlparse
from thrift.transport import TTransport, TSocket, TSSLSocket, THttpClient
from thrift.protocol.TBinaryProtocol import TBinaryProtocol

from rpc_client.funicorn import FunicornService
from rpc_client.funicorn.ttypes import *
import numpy as np
import cv2
import sys

In [2]:
pp = pprint.PrettyPrinter(indent=2)
host = '0.0.0.0'
port = 8001
framed = True

socket = TSocket.TSocket(host, port)
transport = TTransport.TFramedTransport(socket)
protocol = TBinaryProtocol(transport)
client = FunicornService.Client(protocol)

In [78]:
def predict_img_bytes(img_bytes):
    transport.open()
    ret = None
    try:
        ret = client.predict_img_bytes(img_bytes)
    except Exception as e:
        print(e)
    finally:
        transport.close()
    return ret

def ping():
    transport.open()
    try:
        client.ping()
    except Exception as e:
        print(e)
    finally:
        transport.close()
        
        
def get_byte_size(img_bytes):
    print(sys.getsizeof(img_bytes)/(1024))

In [4]:
ping()

In [81]:
img_arr = cv2.imread('/home/lap13607-local/Downloads/277ac2725cd1b68fefc0.jpg')
# img_arr = np.ones(shape=(640, 480, 3), dtype=np.uint8)
# img_arr = np.ones(shape=(1280*2, 720*2, 3), dtype=np.uint8)
# img_arr = cv2.resize(img_arr, (640, 480))
ret, img_flatten = cv2.imencode('.jpg', img_arr, params= [cv2.IMWRITE_JPEG_QUALITY, 70])
img_bytes = img_flatten.tobytes()

In [82]:
img_arr.shape

(1280, 960, 3)

In [83]:
print(sys.getsizeof(img_bytes)/(1024))

135.6982421875


In [37]:
img_flatten = np.frombuffer(img_bytes, dtype=np.uint8)

In [38]:
img_flatten

array([255, 216, 255, ..., 163, 255, 217], dtype=uint8)

In [39]:
img_arr_decoded = cv2.imdecode(img_flatten, cv2.IMREAD_ANYCOLOR)

In [40]:
img_arr_decoded.shape

(480, 640, 3)

In [92]:
%timeit predict_img_bytes(img_bytes)

45.2 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
len(img_bytes)

5427

In [131]:
%timeit ret, img_flatten = cv2.imencode('.jpg', img_arr)

4.39 ms ± 508 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
# ~20ms

In [76]:
import pickle

In [77]:
img_bytes = pickle.dumps(img_arr)

In [79]:
get_byte_size(img_bytes)

3600.1923828125
